# DerivaML Features Example

DerivaML is a class library built on the Deriva Scientific Asset management system that is designed to help simplify a number of the basic operations associated with building and testing ML libraries based on common toolkits such as TensorFlow.  This notebook reviews the basic features of the DerivaML library.


In DerivaML, "features" are the way we attach values to objects in the catalog. A feature could be a computed value that serves as input to a ML model, or it could be a label, that is the result of running a model.  A feature can be a controlled vocabulary term, an asset, or a value.

Each feature in the catalog is distinguished by the name of the feature, the identity of the object that the feature is being attached to, and the execution RID of the process that generated the feature value

## Set up Deriva for test case

In [1]:
from fontTools.misc.bezierTools import namedtuple
%load_ext autoreload
%autoreload 2

In [2]:
import builtins
from deriva.core.utils.globus_auth_utils import GlobusNativeLogin
from deriva_ml.deriva_ml_base import ColumnDefinition, BuiltinTypes
from deriva_ml.schema_setup.test_catalog import create_test_catalog, DemoML
from deriva_ml.execution_configuration import ExecutionConfiguration, Workflow, Execution, WorkflowTerm, Term
from IPython.display import display, Markdown
import itertools
from pathlib import Path
import pandas as pd
import tempfile
import random

Set the details for the catalog we want and authenticate to the server if needed.

In [3]:
hostname = 'dev.eye-ai.org'
domain_schema = 'demo-schema'

gnl = GlobusNativeLogin(host=hostname)
if gnl.is_logged_in([hostname]):
    print("You are already logged in.")
else:
    gnl.login([hostname], no_local_server=True, no_browser=True, refresh_tokens=True, update_bdbag_keychain=True)
    print("Login Successful")


You are already logged in.


Create a test catalog and get an instance of the DerivaML class.

In [4]:
test_catalog = create_test_catalog(hostname, domain_schema)
ml_instance = DemoML(hostname, test_catalog.catalog_id)

2024-10-17 22:34:47,494 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2024-10-17 22:34:47,495 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>


A feature is a set of values that are attached to a table in the DerivaML catalog. Instances of features are distingushed from one another by the ID of the execution that produced the feature value. The execution could be the result of a program, or it could be a manual process by which a person defines a set of values

To create a new feature, we need to know the name of the feature, the table to which it is attached, and the set of values that make up the feature.  The values could be terms from a controlled vocabulary, a set of one or more file based assets, or other values, such as integers, or strings. However, use of strings outside of controlled vocabularies is discouraged.

In [6]:
# Prerequests for our feature, which will include a CV term and asset.

# Create a vocabulary and add a term to it to use in our features.
ml_instance.create_vocabulary("SubjectHealth", "A vocab")
ml_instance.add_term("SubjectHealth", "Sick", description="The subject self reports that they are sick")
ml_instance.add_term("SubjectHealth", "Well", description="The subject self reports that they feel well")

ml_instance.create_vocabulary("ImageQuality", "Controlled vocabulary for image quality")
ml_instance.add_term("ImageQuality", "Good", description="The image is good")
ml_instance.add_term("ImageQuality", "Bad", description="The image is bad")

box_asset = ml_instance.create_asset("BoundingBox", comment="A file that contains a cropped version of a image")

# Now lets create and upload a simple asset.
with tempfile.TemporaryDirectory() as tmpdirname:
    assetdir = Path(tmpdirname) / "BoundingBox"
    assetdir.mkdir(parents=True, exist_ok=True)
    for i in range(10):
        with builtins.open(assetdir / f"box{i}.txt", "w") as fp:
            fp.write(f"Hi there {i}")
    ml_instance.upload_assets(assetdir)  # Upload all of the images in the directory to the asset table

In [56]:
# We are going to have three values in our feature, a controlled vocabulary term from the vocabulary FeatureValue, a file asset and 
# an integer value which we will call "TestCol"
ml_instance.create_feature("Health", "Subject",
                                        terms=["SubjectHealth"],
                                        metadata=[ColumnDefinition(name='Scale', type=BuiltinTypes.int2)])

ml_instance.create_feature("BoundingBox", "Image", assets=[box_asset])
ml_instance.create_feature("Quality", "Image", terms=["ImageQuality"])

ValueError: Table Execution_Subject_Health already exists.

In [8]:
display([f.name for f in ml_instance.find_features("Subject")])
display([f.name for f in ml_instance.find_features("Image")])

['Execution_Subject_Health']

['Execution_Image_BoundingBox', 'Execution_Image_Quality']

Now we can add some features to our images.  To streamline the creation of new feature, we create a class that is specific to the arguments required to create it.

In [50]:
ImageQualityFeature = ml_instance.feature_record_class("Image", "Quality")
ImageBoundingboxFeature = ml_instance.feature_record_class("Image", "Quality")
SubjectWellnessFeature= ml_instance.feature_record_class("Subject", "Health")

display(Markdown('### SubjectWellnessFeature'))
display(SubjectWellnessFeature.model_fields)
display(Markdown('### ImageQualityFeature'))
display(ImageQualityFeature.model_fields)
display(Markdown('### ImageBoundingboxFeature'))
display(ImageBoundingboxFeature.model_fields)


### SubjectWellnessFeature

{'Execution': FieldInfo(annotation=str, required=True),
 'Subject': FieldInfo(annotation=str, required=True),
 'Feature_Name': FieldInfo(annotation=str, required=False, default='Health'),
 'SubjectHealth': FieldInfo(annotation=str, required=True),
 'Scale': FieldInfo(annotation=int, required=True)}

### ImageQualityFeature

{'Execution': FieldInfo(annotation=str, required=True),
 'Image': FieldInfo(annotation=str, required=True),
 'Feature_Name': FieldInfo(annotation=str, required=False, default='Quality'),
 'ImageQuality': FieldInfo(annotation=str, required=True)}

### ImageBoundingboxFeature

{'Execution': FieldInfo(annotation=str, required=True),
 'Image': FieldInfo(annotation=str, required=True),
 'Feature_Name': FieldInfo(annotation=str, required=False, default='Quality'),
 'ImageQuality': FieldInfo(annotation=str, required=True)}

Now using TestFeatureClass, we can create some instances of the feature and add it.  We must have a exeuction_rid in order to define the feature.

In [51]:
api_workflow = Workflow(
    name="API Workflow", 
    url="https://github.com/informatics-isi-edu/deriva-ml/blob/main/pyproject.toml",
    workflow_type="API Workflow"
)

config = ExecutionConfiguration(
    execution=Execution(description="Sample Execution"), 
    workflow=api_workflow, 
    workflow_terms=[WorkflowTerm(term=Term.workflow, name="API Driven Workflow", description="A workflow that only uses feature API")],
    description="Our Sample Workflow instance")
configuration_record = ml_instance.initialize_execution(config)
execution_rid = configuration_record.execution_rid

In [55]:
# Get some images to attach the feature value to.
subject_rids = [i['RID'] for i in ml_instance.domain_path.tables['Subject'].entities().fetch()]
image_rids = [i['RID'] for i in ml_instance.domain_path.tables['Image'].entities().fetch()]
asset_rids = [i['RID'] for i in ml_instance.domain_path.tables['BoundingBox'].entities().fetch()]
# Now create a list of features using the feature creation class returned by create_feature.
subject_feature_list = [SubjectWellnessFeature(
    Subject=subject_rid,
    Execution=execution_rid,
    SubjectHealth= ["Well", "Sick"][random.randint(0,1)],
    Scale=random.randint(1, 10)) for subject_rid in subject_rids]

image_quality_feature_list = [ImageQualityFeature(
    Image=image_rid,
    Execution=execution_rid,
    ImageQuality= ["Good", "Bad"][random.randint(0,1)])
        for image_rid in image_rids]

image_boundingbox_feature_list = [ImageBoundingboxFeature(
    Image=image_rid,
    Execution=execution_rid,
    box_asset=asset_rid)
        for image_rid, asset_rid in zip(image_rids, itertools.cycle(asset_rids))]

#ml_instance.add_features(subject_feature_list)
#ml_instance.add_features(image_feature_list)

ValidationError: 1 validation error for ImageFeatureQuality
ImageQuality
  Field required [type=missing, input_value={'Image': '30R', 'Executi...JE', 'box_asset': '38E'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.8/v/missing

In [ ]:
def strip_system(d):
    return {k:v for k,v in d.items() if k not in ['RCT', 'RMT', 'RCB', 'RMB']}
    
pd.DataFrame([strip_system(i) for i in ml_instance.list_feature("Image", "Feature1")])

In [ ]:
test_catalog.delete_ermrest_catalog(really=True)